In [26]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [27]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 16]

In [28]:
station['date'].count()

1035

In [29]:
# ls == 모든 값이 0인 것

ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

(['depart', 'sunrise', 'sunset', 'snowfall', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'snowfall', 'resultdir', 'daytime'])

In [30]:
for i in station.columns:
    count_null = station[i].isna().sum()
    print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

station_nbr : 0 ( 0.0 % )
date : 0 ( 0.0 % )
tmax : 0 ( 0.0 % )
tmin : 0 ( 0.0 % )
tavg : 0 ( 0.0 % )
depart : 1035 ( 100.0 % )
dewpoint : 0 ( 0.0 % )
wetbulb : 0 ( 0.0 % )
heat : 0 ( 0.0 % )
cool : 0 ( 0.0 % )
sunrise : 1035 ( 100.0 % )
sunset : 1035 ( 100.0 % )
codesum : 0 ( 0.0 % )
snowfall : 1035 ( 100.0 % )
preciptotal : 0 ( 0.0 % )
stnpressure : 0 ( 0.0 % )
sealevel : 0 ( 0.0 % )
resultspeed : 0 ( 0.0 % )
resultdir : 7 ( 0.68 % )
avgspeed : 0 ( 0.0 % )
year : 0 ( 0.0 % )
month : 0 ( 0.0 % )
day : 0 ( 0.0 % )
relative_humility : 0 ( 0.0 % )
windchill : 0 ( 0.0 % )
daytime : 1035 ( 100.0 % )
week7 : 0 ( 0.0 % )
weekend : 0 ( 0.0 % )
code_change : 0 ( 0.0 % )
rainY : 0 ( 0.0 % )
otherY : 0 ( 0.0 % )
nothing : 0 ( 0.0 % )


In [31]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [32]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [33]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
211339,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,NaN,4,0,0,0,0,1,45,107,0
211340,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,NaN,4,0,0,0,0,1,45,108,0
211341,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,NaN,4,0,0,0,0,1,45,109,0
211342,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,NaN,4,0,0,0,0,1,45,110,0
211343,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,NaN,4,0,0,0,0,1,45,111,0


In [34]:
station['log1p_units'] = np.log1p(station.units)

In [35]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
211339,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,40.375498,NaN,4,0,0,0,0,1,45,107
211340,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,40.375498,NaN,4,0,0,0,0,1,45,108
211341,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,40.375498,NaN,4,0,0,0,0,1,45,109
211342,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,40.375498,NaN,4,0,0,0,0,1,45,110
211343,16,2014-10-31,53.0,34.0,43.5,NaN,35.0,41.0,21.0,0.0,...,40.375498,NaN,4,0,0,0,0,1,45,111


In [36]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [37]:
df2.to_csv('station16.csv', sep=',', index=False)

In [38]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 있음
['depart', 'sunrise', 'sunset', 'snowfall', 'daytime']


In [39]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

In [40]:
ls, ls1

(['depart', 'sunrise', 'sunset', 'snowfall', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'snowfall', 'resultdir', 'daytime'])

### 1. 변수변환: df2 (log1p_units)

In [19]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + C(daytime) + 0'

In [41]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.597
Method:                 Least Squares   F-statistic:                     2249.
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:15:36   Log-Likelihood:                 33471.
No. Observations:              211344   AIC:                        -6.666e+04
Df Residuals:                  211204   BIC:                        -6.523e+04
Df Model:                         139                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [42]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

6188

In [43]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,1,0.000000
1,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,2,0.000000
2,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,3,0.000000
3,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,4,0.000000
4,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,5,0.000000
5,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,6,0.000000
6,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,7,0.000000
7,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,8,0.000000
8,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,9,2.197225
9,16,2012-01-01,50.0,30.0,40.0,NaN,37.0,40.0,25.0,0.0,...,NaN,6,1,2,1,1,0,14,10,0.000000


In [44]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 1.261e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:19:26   Log-Likelihood:             6.0063e+05
No. Observations:              205156   AIC:                        -1.201e+06
Df Residuals:                  205016   BIC:                        -1.200e+06
Df Model:                         139                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

In [45]:
ls1

['depart', 'sunrise', 'sunset', 'snowfall', 'resultdir', 'daytime']

### 3. VIF 확인

In [46]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
0,inf,tmax
1,inf,tmin
2,inf,tavg
3,1.165961e+06,stnpressure
4,1.145830e+06,sealevel
5,5.948294e+03,windchill
6,2.507198e+03,dewpoint
7,2.343067e+03,wetbulb
8,1.027256e+03,relative_humility
9,1.900435e+02,heat


### 4. VIF 근거, 빼고 다시 Model 돌림.

In [47]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(cool)\
                            + scale(preciptotal) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) +C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 1.348e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:21:52   Log-Likelihood:             6.0063e+05
No. Observations:              205156   AIC:                        -1.201e+06
Df Residuals:                  205025   BIC:                        -1.200e+06
Df Model:                         130                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]       5.009e-05      0

### 5. VIF 재확인 

In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['cool', 'preciptotal', 'resultspeed', 'avgspeed']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,24.920258,avgspeed
1,23.897377,resultspeed
2,1.199907,preciptotal
3,1.093131,cool


### 6. VIF 근거로 제외하고  Model 돌림.

In [49]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(cool)\
                            + scale(preciptotal) + C(year) + C(month) +C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 1.370e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:23:35   Log-Likelihood:             6.0063e+05
No. Observations:              205156   AIC:                        -1.201e+06
Df Residuals:                  205027   BIC:                        -1.200e+06
Df Model:                         128                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]        6.23e-05      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [50]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(rainY),1.0,0.023462,0.023462,139.795789,3.022470e-32
C(weekend),1.0,0.059698,0.059698,355.707887,2.828115e-79
C(store_nbr),1.0,0.061106,0.061106,364.093999,4.252491e-81
C(month),11.0,0.185301,0.016846,100.372941,3.188750e-229
C(year),3.0,9.256960,3.085653,18385.682456,0.000000e+00
C(item_nbr),110.0,2941.745846,26.743144,159347.443380,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [53]:
# OLS - df2_1_1
model2_1 = sm.OLS.from_formula('log1p_units ~ C(year) + C(month) +C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)

result = model2_1.fit()
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 1.391e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:27:17   Log-Likelihood:             6.0063e+05
No. Observations:              205156   AIC:                        -1.201e+06
Df Residuals:                  205029   BIC:                        -1.200e+06
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]       4.349e-05      0

In [54]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [55]:
formula = 'C(year) + C(month) + C(weekend) + C(item_nbr) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.99013256, 0.988091  , 0.98574209, 0.98998727, 0.98901424,
       0.99025732, 0.98742604, 0.98804902, 0.98734554, 0.98869103])